In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Este notebook require Tensorflow 2.X

In [3]:
print(tf.__version__)

2.1.0


### Descargar Extracto der Shakespeare 

In [4]:
jokes = ""

joke_url = "https://raw.githubusercontent.com/Daronspence/One-Liners/master/jokes.txt"
filepath = keras.utils.get_file("test.txt", joke_url)
with open(filepath) as f:
    joke_text = f.read()

jokes += joke_text

joke_url = "http://www.textfiles.com/humor/TAGLINES/taglines.txt"
filepath = keras.utils.get_file("test.txt", joke_url)
with open(filepath) as f:
    joke_text = f.read()
    
jokes += joke_text

joke_url = "https://raw.githubusercontent.com/simonaco/25daysofserverless/master/jokes.txt"
filepath = keras.utils.get_file("test.txt", joke_url)
with open(filepath) as f:
    joke_text = f.read()
    
jokes += joke_text

### Set de caracteres en la obra (FYI):

In [5]:
"".join(sorted(set(jokes.lower())))

'\n !"$%&\',-.01234589:;=?abcdefghijklmnopqrstuvwxyz'

In [6]:
# remove caracteres extranos.

jokes = jokes.replace("\n", " ")
jokes = jokes.replace("$", " ")
jokes = jokes.replace('"', " ")
jokes = jokes.replace('%', " ")
jokes = jokes.replace('&', " ")
jokes = jokes.replace("'", " ")
jokes = jokes.replace("-", " ")
jokes = jokes.replace(";", "")
jokes = jokes.replace(":", "")
jokes = jokes.replace("=", "")
jokes = jokes.replace(",", "")
                      
"".join(sorted(set(jokes.lower())))

' !.01234589?abcdefghijklmnopqrstuvwxyz'

### Tokenizacion del texto con Keras

In [7]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(jokes)

# probamos la palabra
tokenizer.texts_to_sequences(["speak"])

[[8, 21, 2, 5, 24]]

In [8]:
# podemos converir esa secuencia en texto:
tokenizer.sequences_to_texts([[8, 21, 2, 5, 24]])

['s p e a k']

In [9]:
# numero de caracteres distintos
max_id = len(tokenizer.word_index) 

# cantidad total del caracteres
dataset_size = tokenizer.document_count

print("Elementos distintos",max_id)
print("Cantidad de elementos",dataset_size)

Elementos distintos 38
Cantidad de elementos 142305


### Creamos el Train-Set usando las Funciones de NLP (ver Notebook #1)

In [10]:
# se convierte TODO el texto en secuencias.
[encoded] = np.array(tokenizer.texts_to_sequences([jokes])) - 1

# se genera un train-set del 90% de las secuencias
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [11]:
# se define que las cadenas son de 100 caracteres con 1 caracter de shift
n_steps = 100
window_length = n_steps + 1 # tamano de la ventana
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

# se generan las cadenas planas
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [12]:
# generamos el x_train y y_train usando el mismo codigo del notebook #1
np.random.seed(42)
tf.random.set_seed(42)
batch_size = 32

encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []

for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)

dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda x_train, y_train: (tf.one_hot(x_train, depth=max_id), y_train))
dataset = dataset.prefetch(1)

# se revisan los tamanos de los tensores generados
for x, y in dataset.take(1):
    print(x.shape, y.shape)

(32, 100, 38) (32, 100)


In [13]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

### Modelo Stateful RNN-GRU y Entrenamiento

In [14]:
# arquitecturta
model = keras.models.Sequential([
    keras.layers.GRU(1024, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(512, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation="softmax"))
])

# compilacion y entrenamiento
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
steps_per_epoch = train_size // batch_size // n_steps
history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=400,
                    callbacks=[ResetStatesCallback()])

Train for 40 steps
Epoch 1/400
40/40 [==============================] - 19s 473ms/step - loss: 2.9607
Epoch 2/400
40/40 [==============================] - 14s 348ms/step - loss: 2.7389
Epoch 3/400
40/40 [==============================] - 14s 347ms/step - loss: 2.4859
Epoch 4/400
40/40 [==============================] - 14s 348ms/step - loss: 2.3197
Epoch 5/400
40/40 [==============================] - 14s 347ms/step - loss: 2.2179
Epoch 6/400
40/40 [==============================] - 14s 349ms/step - loss: 2.1257
Epoch 7/400
40/40 [==============================] - 14s 347ms/step - loss: 2.0405
Epoch 8/400
40/40 [==============================] - 14s 348ms/step - loss: 1.9739
Epoch 9/400
40/40 [==============================] - 14s 347ms/step - loss: 1.9104
Epoch 10/400
40/40 [==============================] - 14s 348ms/step - loss: 1.8438
Epoch 11/400
40/40 [==============================] - 14s 349ms/step - loss: 1.7931
Epoch 12/400
40/40 [==============================] - 14s 350ms/st

40/40 [==============================] - 14s 347ms/step - loss: 0.1549
Epoch 195/400
40/40 [==============================] - 14s 347ms/step - loss: 0.1545
Epoch 196/400
40/40 [==============================] - 14s 348ms/step - loss: 0.1449
Epoch 197/400
40/40 [==============================] - 14s 347ms/step - loss: 0.1531
Epoch 198/400
40/40 [==============================] - 14s 347ms/step - loss: 0.1489
Epoch 199/400
40/40 [==============================] - 14s 349ms/step - loss: 0.1500
Epoch 200/400
40/40 [==============================] - 14s 348ms/step - loss: 0.1500
Epoch 201/400
40/40 [==============================] - 14s 347ms/step - loss: 0.1530
Epoch 202/400
40/40 [==============================] - 14s 348ms/step - loss: 0.1522
Epoch 203/400
40/40 [==============================] - 14s 349ms/step - loss: 0.1469
Epoch 204/400
40/40 [==============================] - 14s 348ms/step - loss: 0.1463
Epoch 205/400
40/40 [==============================] - 14s 350ms/step - loss: 0

40/40 [==============================] - 14s 353ms/step - loss: 0.0831
Epoch 387/400
40/40 [==============================] - 14s 353ms/step - loss: 0.0834
Epoch 388/400
40/40 [==============================] - 14s 352ms/step - loss: 0.0860
Epoch 389/400
40/40 [==============================] - 14s 355ms/step - loss: 0.0826
Epoch 390/400
40/40 [==============================] - 14s 352ms/step - loss: 0.0837
Epoch 391/400
40/40 [==============================] - 15s 363ms/step - loss: 0.0832
Epoch 392/400
40/40 [==============================] - 14s 351ms/step - loss: 0.0829
Epoch 393/400
40/40 [==============================] - 14s 353ms/step - loss: 0.0851
Epoch 394/400
40/40 [==============================] - 14s 348ms/step - loss: 0.0806
Epoch 395/400
40/40 [==============================] - 14s 348ms/step - loss: 0.0765
Epoch 396/400
40/40 [==============================] - 14s 349ms/step - loss: 0.0835
Epoch 397/400
40/40 [==============================] - 14s 353ms/step - loss: 0

In [15]:
# esto es un super hack!
stateless_model = keras.models.Sequential([
    keras.layers.GRU(1024, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(512, return_sequences=True),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

stateless_model.build(tf.TensorShape([None, None, max_id]))
stateless_model.set_weights(model.get_weights())

In [16]:
model = stateless_model

# Guardar el Modelo
model.save('jokes.h5')